In [1]:
# pandas and numpy
import numpy as np 
import pandas as pd 
# import data
import sqlite3
# list seq and union
from typing import List, Sequence, Union
# TA-Lib
import talib as ta
# pyechart
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Kline, Line, Bar, Grid

# Get Data

In [2]:
import baostock as bs
import pandas as pd

#### 登陆系统 ####
lg = bs.login()

#### 获取沪深A股历史K线数据 ####
# 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。“分钟线”不包含指数。
# 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
# 周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg
rs = bs.query_history_k_data_plus("sz.300274",
    "date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,isST",
    frequency="d", adjustflag="3")
print('query_history_k_data_plus respond error_code:'+rs.error_code)
print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)

#### 打印结果集 ####
data_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)

#### 结果集输出到csv文件 ####   
result.to_csv("D:\\history_A_stock_k_data.csv", index=False)

#### 登出系统 ####
bs.logout()

login success!
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!


In [3]:
# Get Dataframe
df_values = result[['open','close','low','high']].values.tolist()
timelist = result.date.tolist()

In [4]:
# calculate MA
def calculate_ma(day_count: int, data):
    result: List[Union[float, str]] = []
    for i in range(len(data["values"])):
        if i < day_count:
            result.append("-")
            continue
        sum_total = 0.0
        for j in range(day_count):
            sum_total += float(data["values"][i - j][1])
        result.append(abs(float("%.3f" % (sum_total / day_count))))
    return result

In [5]:
# MAs
types=['SMA','EMA']
for i in range(len(types)):
    result['{}-50'.format(types[i])] = ta.MA(result.close, timeperiod=50, matype=i)
result.fillna('-',inplace=True)
# ma50
sma50 = result['SMA-50'].tolist()

# Plot

In [7]:
import requests
from typing import List, Union

from pyecharts import options as opts
from pyecharts.charts import Kline, Line, Bar, Grid
# K线
Kline = (
    Kline()
    .add_xaxis(xaxis_data = timelist)
    .add_yaxis(
        series_name = "",
        y_axis = df_values,
        # itemstyle_opts = opts.ItemStyleOpts(color='#089981', color0='#f23645'),
        itemstyle_opts=opts.ItemStyleOpts(
            color = "#089981", 
            color0 = "#f23645",
            border_color = "#089981",
            border_color0 = "#f23645"),
    )
    .set_global_opts(
            legend_opts=opts.LegendOpts(
                is_show=True, pos_left="left"
            ),
            datazoom_opts=[
                opts.DataZoomOpts(
                    is_show=False,
                    type_="inside",
                    xaxis_index=[0, 1],
                    range_start=98,
                    range_end=100,
                ),
                opts.DataZoomOpts(
                    is_show=True,
                    xaxis_index=[0, 1],
                    type_="slider",
                    pos_top="85%",
                    range_start=98,
                    range_end=100,
                ),
            ],
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitarea_opts=opts.SplitAreaOpts(
                    is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                ),
            ),
            tooltip_opts=opts.TooltipOpts(
                trigger="axis",
                axis_pointer_type="cross",
                background_color="rgba(245, 245, 245, 0.8)",
                border_width=1,
                border_color="#ccc",
                textstyle_opts=opts.TextStyleOpts(color="#000"),
            ),
            visualmap_opts=opts.VisualMapOpts(
                is_show=False,
                dimension=2,
                series_index=5,
                is_piecewise=True,
                pieces=[
                    {"value": 1, "color": "#00da3c"},
                    {"value": -1, "color": "#ec0000"},
                ],
            ),
            axispointer_opts=opts.AxisPointerOpts(
                is_show=True,
                link=[{"xAxisIndex": "all"}],
                label=opts.LabelOpts(background_color="#777"),
            ),
            brush_opts=opts.BrushOpts(
                x_axis_index="all",
                brush_link="all",
                out_of_brush={"colorAlpha": 0.1},
                brush_type="lineX",
            ),
        )
)

# 简单易懂均线 MA 
line = (
        Line()
        .add_xaxis(xaxis_data = timelist)
        # .add_yaxis(
        #     series_name="MA10",
        #     y_axis=calculate_ma(day_count=5, data=chart_data),
        #     is_smooth=True,
        #     is_hover_animation=False,
        #     is_symbol_show = False,
        #     linestyle_opts=opts.LineStyleOpts(width=3, opacity=0.5),
        #     label_opts=opts.LabelOpts(is_show=False),
        # )
        .add_yaxis(
            series_name = "MA50",
            y_axis = sma50,
            is_smooth = True,
            is_hover_animation = False,
            is_symbol_show = False,
            is_connect_nones=True,
            linestyle_opts = opts.LineStyleOpts(width=1.5),
            label_opts = opts.LabelOpts(is_show=True),
        )
        # .add_yaxis(
        #     series_name="MA150",
        #     y_axis=calculate_ma(day_count=20, data=chart_data),
        #     is_smooth=True,
        #     is_hover_animation=False,
        #     is_symbol_show = False,
        #     linestyle_opts=opts.LineStyleOpts(width=3, opacity=0.5),
        #     label_opts=opts.LabelOpts(is_show=False),
        # )
        # .add_yaxis(
        #     series_name="MA200",
        #     y_axis=calculate_ma(day_count=30, data=chart_data),
        #     is_smooth=True,
        #     is_hover_animation=False,
        #     is_symbol_show = False,
        #     linestyle_opts=opts.LineStyleOpts(width=3, opacity=0.5),
        #     label_opts=opts.LabelOpts(is_show=False),
        # )
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                type_="category",
                grid_index=1,
                axislabel_opts=opts.LabelOpts(is_show=False),
            ),
            yaxis_opts=opts.AxisOpts(
                grid_index=1,
                split_number=3,
                axisline_opts=opts.AxisLineOpts(is_on_zero=False),
                axistick_opts=opts.AxisTickOpts(is_show=False),
                splitline_opts=opts.SplitLineOpts(is_show=False),
                axislabel_opts=opts.LabelOpts(is_show=True),
            ),
        )
    )

# 交易量 volume
bar = (
        Bar()
        .add_xaxis(xaxis_data = timelist)
        .add_yaxis(
            series_name="Volume",
            # 交易量
            y_axis=volumelist,
            xaxis_index=1,
            yaxis_index=1,
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                type_="category",
                is_scale=True,
                grid_index=1,
                boundary_gap=False,
                axisline_opts=opts.AxisLineOpts(is_on_zero=False),
                axistick_opts=opts.AxisTickOpts(is_show=False),
                splitline_opts=opts.SplitLineOpts(is_show=False),
                axislabel_opts=opts.LabelOpts(is_show=False),
                split_number=20,
                min_="dataMin",
                max_="dataMax",
            ),
            yaxis_opts=opts.AxisOpts(
                grid_index=1,
                is_scale=True,
                split_number=2,
                axislabel_opts=opts.LabelOpts(is_show=False),
                axisline_opts=opts.AxisLineOpts(is_show=False),
                axistick_opts=opts.AxisTickOpts(is_show=False),
                splitline_opts=opts.SplitLineOpts(is_show=False),
            ),
            legend_opts=opts.LegendOpts(is_show=False),
        )
    )

# Kline And Line
overlap_kline_line = Kline.overlap(line)

# Grid Overlap + Bar
grid_chart = Grid(
    init_opts=opts.InitOpts(
        width="1000px",
        height="800px",
        animation_opts=opts.AnimationOpts(animation=False),
    )
)

grid_chart.add_js_funcs("var barData = {}".format(timelist))

grid_chart.add(
    overlap_kline_line,
    grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", height="50%"),
)
grid_chart.add(
    bar,
    grid_opts=opts.GridOpts(
        pos_left="10%", pos_right="8%", pos_top="63%", height="16%"
    ),
)

grid_chart.render("btc_pyechart.html")
grid_chart.render_notebook()